##### Copyright 2018 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Basic regression: Predict fuel efficiency

基本回歸：預測燃油效率

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/regression"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/regression.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/regression.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/regression.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In a *regression* problem, the aim is to predict the output of a continuous value, like a price or a probability. Contrast this with a *classification* problem, where the aim is to select a class from a list of classes (for example, where a picture contains an apple or an orange, recognizing which fruit is in the picture).

This tutorial uses the classic [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg) dataset and demonstrates how to build models to predict the fuel efficiency of the late-1970s and early 1980s automobiles. To do this, you will provide the models with a description of many automobiles from that time period. This description includes attributes like cylinders, displacement, horsepower, and weight.

This example uses the Keras API. (Visit the Keras [tutorials](https://www.tensorflow.org/tutorials/keras) and [guides](https://www.tensorflow.org/guide/keras) to learn more.)

在*回歸*問題中，目標是預測連續值的輸出，例如價格或概率。 將此與 *分類* 問題進行對比，其目的是從類別列表中選擇一個類別（例如，圖片包含蘋果或橙子，識別圖片中是哪種水果）。

本教程使用經典的 [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg) 數據集，並演示如何構建模型來預測 20 世紀 70 年代末和 80 年代初的燃油效率 汽車。 為此，您將為模型提供那個時期的許多汽車的描述。 此描述包括氣缸、排量、馬力和重量等屬性。

此示例使用 Keras API。 （訪問 Keras [教程](https://www.tensorflow.org/tutorials/keras) 和 [指南](https://www.tensorflow.org/guide/keras) 以了解更多信息。）

In [3]:
# Use seaborn for pairplot.
!pip install -q seaborn

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [5]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.11.0


## The Auto MPG dataset

The dataset is available from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/).

該數據集可從 [UCI 機器學習庫](https://archive.ics.uci.edu/ml/) 獲得。


### Get the data

First download and import the dataset using pandas:

In [6]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [7]:
dataset = raw_dataset.copy()
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


### Clean the data

The dataset contains a few unknown values:

In [8]:
dataset.isna().sum()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64

Drop those rows to keep this initial tutorial simple:

In [9]:
dataset = dataset.dropna()

The `"Origin"` column is categorical, not numeric. So the next step is to one-hot encode the values in the column with [pd.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html).

Note: You can set up the `tf.keras.Model` to do this kind of transformation for you but that's beyond the scope of this tutorial. Check out the [Classify structured data using Keras preprocessing layers](../structured_data/preprocessing_layers.ipynb) or [Load CSV data](../load_data/csv.ipynb) tutorials for examples.

`"Origin"` 列是分類的，而不是數字的。 因此，下一步是使用 [pd.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) 對列中的值進行一次性編碼。

注意：您可以設置 `tf.keras.Model` 來為您進行這種轉換，但這超出了本教程的範圍。 查看[使用 Keras 預處理層對結構化數據進行分類](../structured_data/preprocessing_layers.ipynb) 或 [加載 CSV 數據](../load_data/csv.ipynb) 教程以獲取示例。

In [10]:
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [11]:
dataset = pd.get_dummies(dataset, columns=['Origin'], prefix='', prefix_sep='')
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan,USA
393,27.0,4,140.0,86.0,2790.0,15.6,82,0,0,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,1,0,0
395,32.0,4,135.0,84.0,2295.0,11.6,82,0,0,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,0,0,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,0,0,1


### Split the data into training and test sets

Now, split the dataset into a training set and a test set. You will use the test set in the final evaluation of your models.

### 將數據拆分為訓練集和測試集

現在，將數據集拆分為訓練集和測試集。 您將在模型的最終評估中使用測試集。

In [12]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

### Inspect the data

Review the joint distribution of a few pairs of columns from the training set.

The top row suggests that the fuel efficiency (MPG) is a function of all the other parameters. The other rows indicate they are functions of each other.

### 檢查數據

查看訓練集中幾對列的聯合分佈。

第一行表明燃油效率 (MPG) 是所有其他參數的函數。 其他行表示它們是彼此的函數。

In [13]:
sns.pairplot(train_dataset[['MPG', 'Cylinders', 'Displacement', 'Weight']], diag_kind='kde')

Let's also check the overall statistics. Note how each feature covers a very different range:

讓我們也檢查一下總體統計數據。 請注意每個功能如何涵蓋非常不同的範圍：

In [14]:
train_dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
MPG,314.0,23.310510,7.728652,10.0,17.00,22.0,28.95,46.6
Cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
Displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
Horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
Weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
Acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
Model Year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0


### Split features from labels

Separate the target value—the "label"—from the features. This label is the value that you will train the model to predict.

### 從標籤中分離特徵

將目標值（“標籤”）與特徵分開。 此標籤是您將訓練模型預測的值。

In [15]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

## Normalization

In the table of statistics it's easy to see how different the ranges of each feature are:

## 歸一化

在統計表中很容易看出每個特徵的範圍有多麼不同：

In [16]:
train_dataset.describe().transpose()[['mean', 'std']]

,mean,std
MPG,23.310510,7.728652
Cylinders,5.477707,1.699788
Displacement,195.318471,104.331589
Horsepower,104.869427,38.096214
Weight,2990.251592,843.898596
Acceleration,15.559236,2.789230
Model Year,75.898089,3.675642
Europe,0.178344,0.383413
Japan,0.197452,0.398712
USA,0.624204,0.485101


It is good practice to normalize features that use different scales and ranges.

One reason this is important is because the features are multiplied by the model weights. So, the scale of the outputs and the scale of the gradients are affected by the scale of the inputs.

Although a model *might* converge without feature normalization, normalization makes training much more stable.

Note: There is no advantage to normalizing the one-hot features—it is done here for simplicity. For more details on how to use the preprocessing layers, refer to the [Working with preprocessing layers](https://www.tensorflow.org/guide/keras/preprocessing_layers) guide and the [Classify structured data using Keras preprocessing layers](../structured_data/preprocessing_layers.ipynb) tutorial.

規範化使用不同尺度和範圍的特徵是一種很好的做法。

這很重要的一個原因是因為特徵乘以模型權重。 因此，輸出的規模和梯度的規模受輸入規模的影響。

儘管模型*可能*在沒有特徵歸一化的情況下收斂，但歸一化使訓練更加穩定。

注意：歸一化 one-hot 特徵沒有任何優勢——為了簡單起見，這裡這樣做了。 有關如何使用預處理層的更多詳細信息，請參閱[使用預處理層](https://www.tensorflow.org/guide/keras/preprocessing_layers)指南和[使用 Keras 預處理層對結構化數據進行分類]( ../structured_data/preprocessing_layers.ipynb) 教程。

### The Normalization layer

The `tf.keras.layers.Normalization` is a clean and simple way to add feature normalization into your model.

The first step is to create the layer:

### 規範化層

`tf.keras.layers.Normalization` 是一種將特徵標準化添加到模型中的干淨簡單的方法。

第一步是創建圖層：

In [17]:
normalizer = tf.keras.layers.Normalization(axis=-1)

Then, fit the state of the preprocessing layer to the data by calling `Normalization.adapt`:

然後，通過調用 Normalization.adapt 將預處理層的狀態與數據相匹配：

In [18]:
normalizer.adapt(np.array(train_features))

Calculate the mean and variance, and store them in the layer:

計算均值和方差，並將它們存儲在圖層中：

In [19]:
print(normalizer.mean.numpy())

[[   5.478  195.318  104.869 2990.252   15.559   75.898    0.178    0.197
     0.624]]


When the layer is called, it returns the input data, with each feature independently normalized:

當層被調用時，它返回輸入數據，每個特徵獨立歸一化：

In [20]:
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

First example: [[   4.    90.    75.  2125.    14.5   74.     0.     0.     1. ]]

Normalized: [[-0.87 -1.01 -0.79 -1.03 -0.38 -0.52 -0.47 -0.5   0.78]]


## Linear regression

Before building a deep neural network model, start with linear regression using one and several variables.

## 線性回歸

在構建深度神經網絡模型之前，首先使用一個和多個變量進行線性回歸。

### Linear regression with one variable

Begin with a single-variable linear regression to predict `'MPG'` from `'Horsepower'`.

Training a model with `tf.keras` typically starts by defining the model architecture. Use a `tf.keras.Sequential` model, which [represents a sequence of steps](https://www.tensorflow.org/guide/keras/sequential_model).

There are two steps in your single-variable linear regression model:

- Normalize the `'Horsepower'` input features using the `tf.keras.layers.Normalization` preprocessing layer.
- Apply a linear transformation ($y = mx+b$) to produce 1 output using a linear layer (`tf.keras.layers.Dense`).

The number of _inputs_ can either be set by the `input_shape` argument, or automatically when the model is run for the first time.

### 單變量線性回歸

從單變量線性回歸開始，根據“馬力”預測“MPG”。

使用“tf.keras”訓練模型通常從定義模型架構開始。 使用“tf.keras.Sequential”模型，它[表示一系列步驟](https://www.tensorflow.org/guide/keras/sequential_model)。

單變量線性回歸模型有兩個步驟：

- 使用“tf.keras.layers.Normalization”預處理層對“馬力”輸入特徵進行歸一化。
- 應用線性變換 ($y = mx+b$) 使用線性層 (`tf.keras.layers.Dense`) 產生 1 個輸出。

_inputs_ 的數量可以通過“input_shape”參數設置，也可以在模型首次運行時自動設置。

First, create a NumPy array made of the `'Horsepower'` features. Then, instantiate the `tf.keras.layers.Normalization` and fit its state to the `horsepower` data:

首先，創建一個由“Horsepower”特徵組成的 NumPy 數組。 然後，實例化 `tf.keras.layers.Normalization` 並將其狀態與 `horsepower` 數據相匹配：

In [21]:
horsepower = np.array(train_features['Horsepower'])

horsepower_normalizer = layers.Normalization(input_shape=[1,], axis=None)
horsepower_normalizer.adapt(horsepower)

Build the Keras Sequential model:

構建 Keras 順序模型：

In [22]:
horsepower_model = tf.keras.Sequential([
    horsepower_normalizer,
    layers.Dense(units=1)
])

horsepower_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 1)                3         
 ion)                                                            
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 5
Trainable params: 2
Non-trainable params: 3
_________________________________________________________________


This model will predict `'MPG'` from `'Horsepower'`.

Run the untrained model on the first 10 'Horsepower' values. The output won't be good, but notice that it has the expected shape of `(10, 1)`:

該模型將從“馬力”預測“MPG”。

在前 10 個“馬力”值上運行未經訓練的模型。 輸出不會很好，但請注意它具有預期的“(10, 1)”形狀：

In [23]:
horsepower_model.predict(horsepower[:10])

1/1 [==============================] - 0s 49ms/step


array([[ 1.02 ],
       [ 0.576],
       [-1.883],
       [ 1.43 ],
       [ 1.293],
       [ 0.508],
       [ 1.532],
       [ 1.293],
       [ 0.337],
       [ 0.576]], dtype=float32)

Once the model is built, configure the training procedure using the Keras `Model.compile` method. The most important arguments to compile are the `loss` and the `optimizer`, since these define what will be optimized (`mean_absolute_error`) and how (using the `tf.keras.optimizers.Adam`).

模型構建完成後，使用 Keras 的“Model.compile”方法配置訓練過程。 編譯最重要的參數是“loss”和“optimizer”，因為它們定義了將優化的內容（“mean_absolute_error”）以及如何優化（使用“tf.keras.optimizers.Adam”）。

In [24]:
horsepower_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

Use Keras `Model.fit` to execute the training for 100 epochs:

使用 Keras `Model.fit` 執行 100 個時期的訓練：

In [25]:
%%time
history = horsepower_model.fit(
    train_features['Horsepower'],
    train_labels,
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

CPU times: user 1.93 s, sys: 422 ms, total: 2.36 s
Wall time: 2.16 s


Visualize the model's training progress using the stats stored in the `history` object:

使用存儲在“歷史”對像中的統計數據可視化模型的訓練進度：

In [26]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,val_loss,epoch
95,3.804361,4.201423,95
96,3.806473,4.157904,96
97,3.808895,4.188793,97
98,3.801918,4.185138,98
99,3.802608,4.193645,99


In [27]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

In [28]:
plot_loss(history)

Collect the results on the test set for later:

收集測試集上的結果以備後用：

In [29]:
test_results = {}

test_results['horsepower_model'] = horsepower_model.evaluate(
    test_features['Horsepower'],
    test_labels, verbose=0)

Since this is a single variable regression, it's easy to view the model's predictions as a function of the input:

由於這是單變量回歸，因此很容易將模型的預測視為輸入的函數：

In [30]:
x = tf.linspace(0.0, 250, 251)
y = horsepower_model.predict(x)

8/8 [==============================] - 0s 599us/step


In [31]:
def plot_horsepower(x, y):
  plt.scatter(train_features['Horsepower'], train_labels, label='Data')
  plt.plot(x, y, color='k', label='Predictions')
  plt.xlabel('Horsepower')
  plt.ylabel('MPG')
  plt.legend()

In [32]:
plot_horsepower(x, y)

### Linear regression with multiple inputs

### 多輸入線性回歸

You can use an almost identical setup to make predictions based on multiple inputs. This model still does the same $y = mx+b$ except that $m$ is a matrix and $b$ is a vector.

Create a two-step Keras Sequential model again with the first layer being `normalizer` (`tf.keras.layers.Normalization(axis=-1)`) you defined earlier and adapted to the whole dataset:

您可以使用幾乎相同的設置來根據多個輸入進行預測。 該模型仍然執行相同的 $y = mx+b$，只是 $m$ 是矩陣而 $b$ 是向量。

再次創建一個兩步 Keras 順序模型，第一層是您之前定義並適應整個數據集的`normalizer` (`tf.keras.layers.Normalization(axis=-1)`)：

In [33]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

When you call `Model.predict` on a batch of inputs, it produces `units=1` outputs for each example:

當您對一批輸入調用 `Model.predict` 時，它會為每個示例生成 `units=1` 輸出：

In [34]:
linear_model.predict(train_features[:10])

1/1 [==============================] - 0s 28ms/step


array([[ 0.084],
       [ 0.285],
       [-1.673],
       [ 0.534],
       [ 1.945],
       [-0.645],
       [ 1.841],
       [ 1.819],
       [-0.753],
       [ 1.34 ]], dtype=float32)

When you call the model, its weight matrices will be built—check that the `kernel` weights (the $m$ in $y=mx+b$) have a shape of `(9, 1)`:

當您調用模型時，將構建其權重矩陣——檢查“內核”權重（$y=mx+b$ 中的 $m$）的形狀是否為“(9, 1)”：

In [35]:
linear_model.layers[1].kernel

<tf.Variable 'dense_1/kernel:0' shape=(9, 1) dtype=float32, numpy=
array([[-0.567],
       [-0.548],
       [-0.512],
       [ 0.753],
       [-0.286],
       [ 0.178],
       [ 0.027],
       [ 0.214],
       [-0.631]], dtype=float32)>

Configure the model with Keras `Model.compile` and train with `Model.fit` for 100 epochs:

使用 Keras `Model.compile` 配置模型並使用 `Model.fit` 訓練 100 個時期：

In [36]:
linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [37]:
%%time
history = linear_model.fit(
    train_features,
    train_labels,
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

CPU times: user 1.73 s, sys: 484 ms, total: 2.21 s
Wall time: 2.04 s


Using all the inputs in this regression model achieves a much lower training and validation error than the `horsepower_model`, which had one input:

使用此回歸模型中的所有輸入可實現比具有一個輸入的“horsepower_model”低得多的訓練和驗證錯誤：

In [38]:
plot_loss(history)

Collect the results on the test set for later:

收集測試集上的結果以備後用：

In [39]:
test_results['linear_model'] = linear_model.evaluate(
    test_features, test_labels, verbose=0)

## Regression with a deep neural network (DNN)

In the previous section, you implemented two linear models for single and multiple inputs.

Here, you will implement single-input and multiple-input DNN models.

The code is basically the same except the model is expanded to include some "hidden" non-linear layers. The name "hidden" here just means not directly connected to the inputs or outputs.

在上一節中，您為單輸入和多輸入實現了兩個線性模型。

在這裡，您將實現單輸入和多輸入 DNN 模型。

除了模型被擴展以包含一些“隱藏的”非線性層之外，代碼基本相同。 這裡的“隱藏”只是指不直接連接到輸入或輸出。

These models will contain a few more layers than the linear model:

* The normalization layer, as before (with `horsepower_normalizer` for a single-input model and `normalizer` for a multiple-input model).
* Two hidden, non-linear, `Dense` layers with the ReLU (`relu`) activation function nonlinearity.
* A linear `Dense` single-output layer.

Both models will use the same training procedure, so the `compile` method is included in the `build_and_compile_model` function below.

這些模型將包含比線性模型更多的層：

* 歸一化層，和以前一樣（單輸入模型使用“horsepower_normalizer”，多輸入模型使用“normalizer”）。
* 兩個隱藏的、非線性的、具有 ReLU (`relu`) 非線性激活函數的“密集”層。
* 線性“密集”單輸出層。

兩個模型將使用相同的訓練過程，因此 `compile` 方法包含在下面的 `build_and_compile_model` 函數中。

In [40]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

### Regression using a DNN and a single input

Create a DNN model with only `'Horsepower'` as input and `horsepower_normalizer` (defined earlier) as the normalization layer:

創建一個僅將“Horsepower”作為輸入並將“horsepower_normalizer”（之前定義的）作為歸一化層的 DNN 模型：

In [41]:
dnn_horsepower_model = build_and_compile_model(horsepower_normalizer)

This model has quite a few more trainable parameters than the linear models:

這個模型比線性模型有更多的可訓練參數：

In [42]:
dnn_horsepower_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 1)                3         
 ion)                                                            
                                                                 
 dense_2 (Dense)             (None, 64)                128       
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,356
Trainable params: 4,353
Non-trainable params: 3
_________________________________________________________________


Train the model with Keras `Model.fit`:

使用 Keras `Model.fit` 訓練模型：

In [43]:
%%time
history = dnn_horsepower_model.fit(
    train_features['Horsepower'],
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

CPU times: user 2.04 s, sys: 518 ms, total: 2.56 s
Wall time: 2.15 s


This model does slightly better than the linear single-input `horsepower_model`:

該模型比線性單輸入“horsepower_model”稍好：

In [44]:
plot_loss(history)

If you plot the predictions as a function of `'Horsepower'`, you should notice how this model takes advantage of the nonlinearity provided by the hidden layers:

如果將預測繪製為“馬力”的函數，您應該注意到該模型如何利用隱藏層提供的非線性：

In [45]:
x = tf.linspace(0.0, 250, 251)
y = dnn_horsepower_model.predict(x)

8/8 [==============================] - 0s 730us/step


In [46]:
plot_horsepower(x, y)

Collect the results on the test set for later:

收集測試集上的結果以備後用：

In [47]:
test_results['dnn_horsepower_model'] = dnn_horsepower_model.evaluate(
    test_features['Horsepower'], test_labels,
    verbose=0)

### Regression using a DNN and multiple inputs

Repeat the previous process using all the inputs. The model's performance slightly improves on the validation dataset.

使用所有輸入重複前面的過程。 該模型的性能在驗證數據集上略有提高。

In [48]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 9)                19        
 n)                                                              
                                                                 
 dense_5 (Dense)             (None, 64)                640       
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,884
Trainable params: 4,865
Non-trainable params: 19
_________________________________________________________________


In [49]:
%%time
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

CPU times: user 2.05 s, sys: 495 ms, total: 2.55 s
Wall time: 2.13 s


In [50]:
plot_loss(history)

Collect the results on the test set:

收集測試集上的結果：

In [51]:
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

## Performance

Since all models have been trained, you can review their test set performance:

由於所有模型都經過訓練，您可以查看它們的測試集性能：

In [52]:
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T

,Mean absolute error [MPG]
horsepower_model,3.642541
linear_model,2.499848
dnn_horsepower_model,2.876769
dnn_model,1.623830


These results match the validation error observed during training.

這些結果與訓練期間觀察到的驗證錯誤相匹配。

### Make predictions

You can now make predictions with the `dnn_model` on the test set using Keras `Model.predict` and review the loss:

作出預測

您現在可以使用 Keras `Model.predict` 在測試集上使用 `dnn_model` 進行預測並查看損失：

In [53]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)


3/3 [==============================] - 0s 1ms/step


It appears that the model predicts reasonably well.

Now, check the error distribution:

看來該模型的預測相當不錯。

現在，檢查錯誤分佈：

In [54]:
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [MPG]')
_ = plt.ylabel('Count')

If you're happy with the model, save it for later use with `Model.save`:

如果您對模型感到滿意，請使用 `Model.save` 保存它供以後使用：

In [55]:
dnn_model.save('dnn_model')

INFO:tensorflow:Assets written to: dnn_model/assets


INFO:tensorflow:Assets written to: dnn_model/assets


If you reload the model, it gives identical output:

如果您重新加載模型，它會給出相同的輸出：

In [56]:
reloaded = tf.keras.models.load_model('dnn_model')

test_results['reloaded'] = reloaded.evaluate(
    test_features, test_labels, verbose=0)

In [57]:
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T

,Mean absolute error [MPG]
horsepower_model,3.642541
linear_model,2.499848
dnn_horsepower_model,2.876769
dnn_model,1.623830
reloaded,1.623830


## Conclusion

This notebook introduced a few techniques to handle a regression problem. Here are a few more tips that may help:

- Mean squared error (MSE) (`tf.keras.losses.MeanSquaredError`) and mean absolute error (MAE) (`tf.keras.losses.MeanAbsoluteError`) are common loss functions used for regression problems. MAE is less sensitive to outliers. Different loss functions are used for classification problems.
- Similarly, evaluation metrics used for regression differ from classification.
- When numeric input data features have values with different ranges, each feature should be scaled independently to the same range.
- Overfitting is a common problem for DNN models, though it wasn't a problem for this tutorial. Visit the [Overfit and underfit](overfit_and_underfit.ipynb) tutorial for more help with this.

## 結論

本筆記本介紹了一些處理回歸問題的技術。 以下是一些可能有幫助的提示：

- 均方誤差 (MSE) (`tf.keras.losses.MeanSquaredError`) 和平均絕對誤差 (MAE) (`tf.keras.losses.MeanAbsoluteError`) 是用於回歸問題的常見損失函數。 MAE 對異常值不太敏感。 不同的損失函數用於分類問題。
- 同樣，用於回歸的評估指標不同於分類。
- 當數字輸入數據特徵具有不同範圍的值時，每個特徵應獨立縮放到相同範圍。
- 過度擬合是 DNN 模型的常見問題，但在本教程中這不是問題。 訪問 [過擬合和欠擬合](overfit_and_underfit.ipynb) 教程以獲得更多幫助。